In [1]:
from sage import *
import hashlib
import os
import binascii

In [2]:
pip install numpy


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install cryptography


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import cryptography


In [5]:
import numpy as np

In [64]:
class TwistedEdwardsCurve:
    def __init__(self, curve_name):
        if curve_name == "edwards25519":
            self.p = 2**255 - 19
            self.E = EllipticCurve(GF(self.p), [0,486662,0,1,0])
            
        elif curve_name == "edwards448":
            self.curve = edwards448()
            self.q = self.curve.field_modulus()
        else:
            raise ValueError("Curva inválida: " + curve_name)
    
    def BasePointED25519(self):
        x = 15112221349535400772501151409588531511454012693041857206046113283949847762202
        y = 46316835694926478169428394003475163141307993866256225615783033603165251855960
        return (x, y)
    

    
    def scalar_multiplication(self,k, P):
        if k == 0:
            return (0, 1)  # Retornar o ponto neutro da curva
        Q = P
        R = None  # R será o ponto resultante
        while k:
            if k & 1:  # Se o bit menos significativo de k é 1
                if R is None:
                    R = Q
                else:
                    R = self.add_point(R, Q)  # Adiciona Q a R
            Q = self.add_point(Q, Q)  # Dobra o ponto Q
            k >>= 1  # Desloca k para a direita em 1 bit
        return R
        
    def encode_pk(self, point):
        x, y = point
        y_bytes = int(y).to_bytes(32, byteorder='little')
        return y_bytes
 
    
    def bytes_to_point(self, b):
    # O campo prime para Edwards25519
        p = 2**255 - 19
    # Coeficiente d na equação da curva Edwards25519
        d = -121665 * pow(121666, -1, p) % p

    # Converter bytes para y, considerando o último bit para a paridade de x
        y = int.from_bytes(b, byteorder='big')
        x_parity = y >> 255
        y &= (1 << 255) - 1  # Limpar o bit de paridade

    # Resolver a equação da curva para x^2
        u = (y * y - 1) % p
        v = (d * y * y + 1) % p
        x2 = u * pow(v, -1, p) % p  # x^2 = u/v mod p

    # Resolver para x usando raízes quadradas modulares
        x = pow(x2, (p + 3) // 8, p)
        if (x * x - x2) % p != 0:
            x = (x * pow(2, (p - 1) // 4, p)) % p

    # Ajustar a paridade de x se necessário
        if (x % 2 != x_parity):
            x = p - x

        return (x, y)

    
    def add_point(self, P, Q):
        x1, y1 = P
        x2, y2 = Q
        p = self.p
        a = -1  # Este é o valor correto de 'a' para Edwards25519.
    
    # O valor de 'd' para Edwards25519. Certifique-se de usar o valor correto.
        d = -121665 * self.inv(121666) % p

    # Calculando os componentes da fórmula de maneira direta e clara
        x3 = (x1*y2 + x2*y1) % p
        y3 = (y1*y2 - a*x1*x2) % p
        denom = (1 + d*x1*x2*y1*y2) % p
    
    # Para evitar a divisão direta, usamos a inversão modular
        denom_inv = self.inv(denom)
    
        x3 = (x3 * denom_inv) % p
        y3 = (y3 * denom_inv) % p

        return (x3, y3)

    def inv(self, x):
    # Utiliza o Teorema de Fermat's Little para a inversão modular.
        return pow(x, self.p-2, self.p)
   
#ss
class EdDSA:
    def __init__(self, curve_name):
        self.curve = TwistedEdwardsCurve(curve_name)
        
    def Hashing(self, m):
        return hashlib.sha512(m).digest()
    
    def HashToInt(self, m):
        h = self.Hashing(m)
        return int.from_bytes(h, byteorder='big')

    def generate_keypair(self):
        sk = os.urandom(32)  # Chave privada: 32 bytes aleatórios
        hash_sk = hashlib.sha512(sk).digest()  # Hash SHA-512 da chave privada
        
        # Clamping: Ajustar bits conforme as especificações Ed25519
        hash_sk = bytearray(hash_sk)
        hash_sk[0] &= 248
        hash_sk[31] &= 127
        hash_sk[31] |= 64
        
        # Gerar a chave pública
        pk_point = self.curve.scalar_multiplication(int.from_bytes(hash_sk[:32], 'little'), self.curve.BasePointED25519())
        pk = self.curve.encode_pk(pk_point)
        return sk, pk

    def sign(self, sk, message):
        hash_sk = self.Hashing(sk)  # SHA-512 sobre a chave privada
    # Converte os primeiros 32 bytes do hash para um inteiro do Python
        s = int.from_bytes(hash_sk[:32], 'little')

    # Aplica clamping para Ed25519 nos bits relevantes de 's'
        s &= 0xF8FFFFFF  # Aplica máscara para os últimos 3 bits de s
        s |= 0x40000000  # Define o bit 255 para 1
        s &= 0x7FFFFFFF  # Define o bit 254 para 0

    # Calcula 'r' usando o hash da parte relevante da chave privada e a mensagem
        r = self.HashToInt(hash_sk[32:] + message)
        R = self.curve.scalar_multiplication(r, self.curve.BasePointED25519())
        R_enc = self.curve.encode_pk(R)
    
    # Obtem a representação de ponto da chave pública para 's'
        pk_point = self.curve.scalar_multiplication(s, self.curve.BasePointED25519())
        pk = self.curve.encode_pk(pk_point)
    
    # Calcula 'k' usando o hash de R, pk e a mensagem
        k = self.HashToInt(R_enc + pk + message)

    # Calcula 'S' como a soma de 'r' e 'k' multiplicado por 's', modulo p
        S = (r + k * s) % self.curve.p

    # Codifica 'S' como bytes
        S_enc = int(S).to_bytes(32, byteorder='little')
    
        return R_enc, S_enc
        
    def verify(self, pk_encoded, R_encoded, S_encoded, message):
        pk = self.curve.bytes_to_point(pk_encoded)
        R = self.curve.bytes_to_point(R_encoded)
        S = int.from_bytes(S_encoded, byteorder='little')

    # Concatena R, pk, e a mensagem para calcular o hash H
        concat = R_encoded + pk_encoded + message
        H = self.HashToInt(concat)

    # Calcula [H]pk + [S]G
        H_pk = self.curve.scalar_multiplication(H, pk)
        S_G = self.curve.scalar_multiplication(S, self.curve.BasePointED25519())
        calculated_R = self.curve.add_point(H_pk, S_G)

    # Compara o ponto R calculado com o ponto R da assinatura
        return print(calculated_R), print(R)


In [65]:
curve_name = "edwards25519"

# Gerando par de chaves
sk, pk = EdDSA(curve_name).generate_keypair()
print("Chave Privada: ", sk)
print("Chave Pública: ", pk)


# Mensagem a ser assinada
message = b"Exemplo de mensagem para assinatura EdDSA"

R, S = EdDSA(curve_name).sign(sk, message)  # Include the message argument here
print("Assinatura: ", R, S)

verify_signature = EdDSA(curve_name).verify(pk, R, S, message)
print(verify_signature)

Chave Privada:  b'\xcd0>\x8e\x18H%\xd5+_T6\xe7\x07\xa8\x83\xdcg\x82\xa4\x7fi\xd5\x03\xcf\x8c\x93T;\x02\xc2\x97'
Chave Pública:  b'\xe8\xb8\x8808\xd6\xd8\xa9u\x12\xf9Q\xbc\xb0x\xf3\x0f5\xda(\xa3D\xd1\xf27aBR\xba\x97wj'
Assinatura:  b'J\xd2\x87>3[X\xc1o\xa1\xc3\xfc?\x0f\x1c\x87\x8f2\x8d\x04(\xaa\xc5Q \x93\xbe)\xe38_"' b'\xba\xa0\x90\xe9\xec\xaa\xda-[\x13\xa8\xa5\xad`c\x87rx7\xa1f\xe0\xcb\x84x1\xb4\x18D7\x17@'
(54201427784555283899487552656633610209751722947273021212296138729141459401671, 38166799778808222227854033218019111359091683236984859697896283013173027764872)
(46879182317340149435384286754696656966262406328861392820454090734533027414248, 33843122091447188266830661114230617319709880553442358328003912402089316736802)
(None, None)


In [8]:
9069033483165579256249444165810986061298147879344115721450937340282206683136
9069033483165579256249444165810986061298147879344115721450937340282206683136
68459919088386530093563380844831333248310053827079694763041459578015187676164

68459919088386530093563380844831333248310053827079694763041459578015187676164